In [2]:
# Install necessary libraries
import os
os.system("pip install langchain==0.3.12 rouge_score langchain-community transformers torch sentence-transformers datasets evaluate")
import asyncio
from transformers import pipeline
from langchain.agents import initialize_agent, Tool
from langchain_community.llms import HuggingFacePipeline
from langchain.tools import tool
from langchain.agents import AgentType
from datasets import load_dataset
import torch  # Import torch for detecting GPU

# 1. Load the Dataset
print("Loading the dataset...")
ds = load_dataset("Ka4on/radiology")

# Shuffle the dataset for randomness
shuffled_train = ds['train'].shuffle(seed=42)  # You can change the seed for different random splits
shuffled_test = ds['test'].shuffle(seed=42)

# Split the shuffled dataset into training, evaluation, and test sets
train_dataset = shuffled_train.select(range(70))  # First 700 samples for training
eval_dataset = shuffled_test.select(range(20))  # First 200 samples for evaluation
test_dataset = shuffled_test.select(range(20, 30))  # Next 100 samples for testing

# Display a sample row for reference
print("\nSample from the Training Set:")
print(f"Instruction: {train_dataset[0]['instruction']}")
print(f"Input: {train_dataset[0]['input'][:500]}...")  # First 500 characters of the input
print(f"Output: {train_dataset[0]['output']}\n")

# 2. Check for GPU Availability
device = 0 if torch.cuda.is_available() else -1  # If GPU is available, use device 0, else use CPU
device_name = "GPU" if device >= 0 else "CPU"
print(f"Using {device_name} for inference.\n")

# 3. Load the Summarization Pipeline with explicit truncation
print("Loading the summarization model...")
summarization_pipeline = pipeline("summarization",
                                  model="facebook/bart-large-cnn",
                                  device=device,  # Pass the device argument to use GPU or CPU
                                  truncation=True,  # Ensure truncation is explicitly enabled
                                  max_length=1024)  # You can adjust the max length as needed

# 4. Define the Summarization Tool
@tool
async def summarize_tool(text: str) -> str:
    """Summarize the input text into a concise format asynchronously."""
    # Truncate the input if it's too long to avoid errors
    max_input_length = 1024  # You can change this depending on the model's input limits
    if len(text.split()) > max_input_length:
        text = ' '.join(text.split()[:max_input_length])  # Truncate to the first 1024 tokens (approx.)

    # Summarize text using the HuggingFace pipeline asynchronously
    summary = await asyncio.to_thread(summarization_pipeline, text, max_length=1000, min_length=30, do_sample=False, truncation=True)
    return summary[0]['summary_text']

# 5. Load the Reasoning LLM for LangChain
print("Loading the reasoning model for the agent...")
generation_pipeline = pipeline("text-generation",
                               model="EleutherAI/gpt-neo-1.3B",
                               device=device,  # Pass the device argument to use GPU or CPU
                               max_length=1200,
                               truncation=True)  # Explicitly enable truncation
llm = HuggingFacePipeline(pipeline=generation_pipeline)

# 6. Initialize the Summarization Tool for LangChain Agent
tools = [
    Tool(
        name="TextSummarizer",
        func=summarize_tool,
        description="Summarizes a given text into a shorter version."
    )
]

# 7. Initialize the LangChain Agent
print("Initializing the LLM agent...")
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 8. Summarization Worker Function (Asynchronous)
async def process_entry(entry):
    try:
        input_text = entry['input']
        # Summarize the text using the agent asynchronously
        summarized_text = await agent.arun(f"Summarize the following text: {input_text}")

        # Ensure the summarized text is valid before returning
        if summarized_text is None or len(summarized_text.strip()) == 0:
            summarized_text = "Summary not available"  # Fallback summary

        return summarized_text
    except Exception as e:
        print(f"Error processing entry: {e}")
        return "Summary failed"  # Ensure some value is returned

# 9. Parallel Processing (Asynchronous)
async def process_dataset(dataset, name="Dataset"):
    print(f"\nProcessing {name} dataset...")

    # Use asyncio.gather to process the dataset in parallel
    tasks = [process_entry(entry) for entry in dataset]
    results = await asyncio.gather(*tasks)

    # Display the results (summaries)
    for i, result in enumerate(results):
        print(f"Entry {i+1}: {result}\n")

# 10. Main Execution for Jupyter/IPython
async def main():
    # Process the datasets (training, evaluation, test)
    await process_dataset(train_dataset, name="Training")
    await process_dataset(eval_dataset, name="Evaluation")
    await process_dataset(test_dataset, name="Test")

# Run the main asynchronous function in Jupyter/IPython environment
if __name__ == "__main__":
    await main()


Loading the dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_set.jsonl:   0%|          | 0.00/110M [00:00<?, ?B/s]

test_set.jsonl:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/135466 [00:00<?, ? examples/s]

...

Error processing entry: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

TextSummarizer(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - Summarizes a given text into a shorter version.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [TextSummarizer]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Summarize the following text: 40 years old Male. Reason: fever

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error processing entry: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

TextSummarizer(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - Summarizes a given text into a shorter version.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [TextSummarizer]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Summarize the following text: 63 years old Female. Reason: sob

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error processing entry: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

TextSummarizer(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - Summarizes a given text into a shorter version.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [TextSummarizer]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Summarize the following text: Other malaise and fatigue [780.7

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [11]:
from transformers import pipeline
import torch
device = 0 if torch.cuda.is_available() else -1  # If GPU is available, use device 0, else use CPU
summarization_pipeline = pipeline("summarization",
                                  model="facebook/bart-large-cnn",
                                  device=device)

# Test the pipeline directly (to check if it's working outside of Gradio)
test_text = "Summarize the following text: Age: 46 yearsGender: MaleReason for Study: Reason: cause of fevers History: Left ventricular aneurysm. Left ventricular dilating with ventricular fibrillation. Left ventricular hypertrophy with dilatation.Left basal and apical atelectasis. Right ventricle systolic heart failure. Left ventricular aneurysm. "  # Make it long enough for summarization
summary = summarization_pipeline(test_text)
print(summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 142, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


[{'summary_text': 'Left ventricular dilating with ventricular fibrillation. Left ventricular hypertrophy with dilatation. Left basal and apical atelectasis. Right ventricle systolic heart failure. Left Ventricular aneurysm. Right Ventricle Systolic Heart failure.'}]


In [12]:
!pip install gradio
import gradio as gr
import asyncio
from transformers import pipeline
import torch

# ... (Your existing code to define summarization_pipeline) ...
#  from previous response, including the 'import torch' line

# Define the Gradio Interface
def gradio_interface(text_input):
    """Interface function for Gradio."""
    # Use the summarization_pipeline directly
    summary = summarization_pipeline(text_input, max_length=1000, min_length=30, do_sample=False, truncation=True)
    # Extract the summary text
    summary_text = summary[0]['summary_text'] if isinstance(summary, list) and len(summary) > 0 else "Summary not available."
    return summary_text

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_interface,  # Function to call for summarization
    inputs=gr.Textbox(label="Input Text", lines=5, placeholder="Enter text here..."),  # Text input
    outputs=gr.Textbox(label="Summary"),  # Output textbox for the summary
    title="Text Summarizer",  # Interface title
    description="Enter a passage of text, and this tool will summarize it for you.",  # Description
    theme="default",  # You can choose a theme, "default" is good for a simple UI
)

# Launch the Gradio interface
if __name__ == "__main__":
    iface.launch(share=True)  # share=True gives you a public URL to access the interface

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://838db98afe9f8c7597.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
!pip install gradio
import gradio as gr
import asyncio
from transformers import pipeline
import torch

# ... (Your existing code to define summarization_pipeline) ...
#  from previous response, including the 'import torch' line

# Define the Gradio Interface
def gradio_interface(text_input):
    """Interface function for Gradio."""
    # Use the summarization_pipeline directly
    summary = summarization_pipeline(text_input, max_length=1000, min_length=30, do_sample=False, truncation=True)
    # Extract the summary text
    summary_text = summary[0]['summary_text'] if isinstance(summary, list) and len(summary) > 0 else "Summary not available."
    return summary_text

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_interface,  # Function to call for summarization
    inputs=gr.Textbox(label="Input Text", lines=5, placeholder="Enter text here..."),  # Text input
    outputs=gr.Textbox(label="Summary"),  # Output textbox for the summary
    title="Radiology Report Summarizer",  # Interface title
    description="Enter a Radiology report text, and this tool will summarize it for you.",  # Description
    theme="default",  # You can choose a theme, "default" is good for a simple UI
)

# Launch the Gradio interface
if __name__ == "__main__":
    iface.launch(share=True)  # share=True gives you a public URL to access the interface

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54f1e7b389fe16e20f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
